In [1]:
import numpy as np
import pandas as pd
from possibilearn import *
from possibilearn.kernel import GaussianKernel
from clustering import *
import math
import random
import matplotlib.pyplot as plt

#sample generator
def g(m):
    return (-4 + np.random.random(4*m) * 8).reshape((m, 4))

def gr_membership_contour(estimated_membership):
    x = np.arange(-4, 4, .1)
    y = np.arange(-4, 4, .1)
    X, Y = np.meshgrid(x, y)
    zs = np.array([estimated_membership((x, y))
                   for x,y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    membership_contour = plt.contour(X, Y, Z,
                                     levels=(.1, .3, .5, .7, .8, .9, .95), colors='k')
    plt.clabel(membership_contour, inline=1)

def gr_dataset(): 
    for lab, col in zip(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'),
                        ('blue', 'green', 'red')):
        plt.scatter(iris_values_4d[iris_labels==lab, 0],
                    iris_values_4d[iris_labels==lab, 1],
                    label=lab,
                    c=col)

def get_different_clusters(clusters, clusters_index, clusters_labels):
    d={}
    c=[]
    l=[]
    i=[]
    for j in range(len(clusters_labels)):
        if clusters_labels[j] not in d:
            d[clusters_labels[j]] = True
            c.append(clusters[j])
            l.append(clusters_labels[j])
            i.append(clusters_index[j])
    return c, i, l
        

source = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

iris_df = pd.read_csv(
    filepath_or_buffer=source,
    header=None,
    sep=',')

iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # drops the empty line at file-end

iris_values = iris_df.iloc[:,:4].values
iris_labels = iris_df.iloc[:,4].values 

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

iris_values_std = StandardScaler().fit_transform(iris_values)

pca_4d = PCA(n_components=4)
iris_values_4d = pca_4d.fit_transform(iris_values_std)

In [2]:
sigmas = np.array([0.230,0.234, 0.232])
cs = np.array([10**i for i in range(-1,2)])
couples = [(c,s) for s in sigmas for c in cs]

In [3]:
n = len(iris_values_4d)
iterations = 10

validation_accuracy={}
test_accuracy={}
training_accuracy={}
valid_iteration = 0
while(valid_iteration < iterations):
    print '\n\nholdout iteration {}'.format(valid_iteration)
    
    #random permutation of train validation and test sets
    permutation = np.random.permutation(range(n))
    perc_train, perc_val, perc_test = (.8, .1, .1)
    train_index = permutation[:int(n*perc_train)]
    validation_index = permutation[int(n*perc_train):int(n*(perc_train+perc_val))]
    test_index = permutation[int(n*(perc_train+perc_val)):]
    train_set = iris_values_4d[[i for i in train_index]]
    validation_set = iris_values_4d[[i for i in validation_index]]
    test_set = iris_values_4d[[i for i in test_index]]
    
    # find the couple that have the best accuracy for this permutation
    best_couple = (-1,-1) #initialization of best couple
    best_accuracy = -1 #initialization of best accuracy
    
    for (c, sigma) in couples:
        print '\nmodel selection: trying parameters c={}, sigma={}'.format(c, sigma)
        print 'starting clusterization'
        try:
            clusters_index, r = clustering(train_set, sigma, c)
            
            print 'clusters ', clusters_index
            
            clusters_index = [cl for cl in clusters_index if len(cl) > 1] #exclude singleton clusters
            
            #sort clusters 
            clusters_index.sort(key = lambda x: -len(x))
            
            #clusters_index = clusters_index[:3]
            
            clusters = [train_set[cl] for cl in clusters_index] #get the clusters with the points
            
            clusters_labels = [pd.Series([iris_labels[train_index[i]] for i in clusters_index[k]])
                          .mode()[0] for k in range(len(clusters))] #associate clusters with labels
            #print 'first labels: ', clusters_labels
            
            clusters, clusters_index, clusters_labels = get_different_clusters(clusters, clusters_index, clusters_labels)
            
            #print 'second labels: ',clusters_labels   
                
                
            #the clusters must be 3 and represent the 3 different labels
            if len(clusters) == 3 and len(set(clusters_labels)) == 3:
                print 'got enough clusters'
                print 'labelled clusters ', clusters_labels
                
                cluster_indices = [clusters_index[cluster_id] for cluster_id in range(3)]
                mus = [[1 if i in cc else 0 for i in range(int(n*perc_train))] for cc in cluster_indices]
                
                estimated_memberships = []
                for i in range(3):
                    print 'inferring membership {}'.format(i)
                    estimated_membership, _ = possibility_learn(train_set,
                                                      mus[i],
                                                      c=c,
                                                      k=GaussianKernel(sigma),
                                                      sample_generator=g)
                    estimated_memberships.append(estimated_membership)
                
                if not (None in estimated_memberships):
                    
                    print 'validating model'

                    guessed_labels = np.array([clusters_labels[np.argmax([e(t) for e in estimated_memberships])] 
                                       for t in validation_set])

                    correct_labels = np.array(iris_labels[validation_index])
                    #print 'correct labels: ', correct_labels
                    #print 'guessed labels: ', guessed_labels
                    accuracy = float(sum(guessed_labels == correct_labels)) / len(validation_set)
                    print 'accuracy', accuracy
                    
                    if accuracy >= best_accuracy:
                        best_accuracy = accuracy
                        best_couple = c, sigma
                    
                else:
                    print 'unable to infer an estimated membership function'
                    continue
                            
            else:
                print 'not enough clusters'
                continue
            
        except ValueError as e:
            print str(e)
            
    c_best, sigma_best = best_couple
    if c_best > 0 and sigma_best > 0:
        
        #with the couple with the best accuracy infer the membership function merging train and validation set
        new_train_set = np.vstack((train_set, validation_set))
        new_train_index = np.hstack((train_index, validation_index))
        
        print '\nstarting training with best couple: ', best_couple 
        print 'starting clusterization'
        
        try:
            clusters_index, r = clustering(new_train_set, sigma, c)
            clusters_index = [cl for cl in clusters_index if len(cl) > 1] #exclude singleton clusters
            
            #sort clusters 
            clusters_index.sort(key = lambda x: -len(x))
            
            #clusters_index = clusters_index[:3]
            clusters = [new_train_set[cl] for cl in clusters_index] #get the clusters with the points
            
            clusters_labels = [pd.Series([iris_labels[new_train_index[i]] for i in clusters_index[k]])
                          .mode()[0] for k in range(len(clusters))] #associate clusters with labels
            
            clusters, clusters_index, clusters_labels = get_different_clusters(clusters, clusters_index, clusters_labels)
                     
            #the clusters must be 3 and represent the 3 different labels
            if len(clusters) == 3 and len(set(clusters_labels)) == 3:
                print 'got enough clusters'
                print 'labelled clusters ', clusters_labels
                
                cluster_indices = [clusters_index[cluster_id] for cluster_id in range(3)]
                mus = [[1 if i in cc else 0 for i in range(int(n*(perc_train+perc_val)))] for cc in cluster_indices]
                
                estimated_memberships = []
                for i in range(3):
                    print 'inferring membership {}'.format(i)
                    estimated_membership, _ = possibility_learn(new_train_set,
                                                      mus[i],
                                                      c=c,
                                                      k=GaussianKernel(sigma),
                                                      sample_generator=g)
                    estimated_memberships.append(estimated_membership)
                    
                print 'validating test'
                if not (None in estimated_memberships):
                    
                    print 'the test is valid'
                    
                    #compute the accuracy on test set
                    guessed_labels = np.array([clusters_labels[np.argmax([e(t) for e in estimated_memberships])] 
                                       for t in test_set])

                    correct_labels = np.array(iris_labels[test_index])
                    #print 'correct labels: ', correct_labels
                    #print 'guessed labels: ', guessed_labels
                    accuracy = float(sum(guessed_labels == correct_labels)) / len(test_set)
                    print 'accuracy of the best couple on test set is ', accuracy
                    
                    test_accuracy[(valid_iteration, c, sigma)] = accuracy
                    
                    #compute the accuracy on training set
                    guessed_labels = np.array([clusters_labels[np.argmax([e(t) for e in estimated_memberships])] 
                                       for t in new_train_set])

                    correct_labels = np.array(iris_labels[new_train_index])
                    #print 'correct labels: ', correct_labels
                    #print 'guessed labels: ', guessed_labels
                    accuracy = float(sum(guessed_labels == correct_labels)) / len(new_train_set)
                    print 'accuracy of the best couple on training set is ', accuracy
                    
                    training_accuracy[(valid_iteration, c, sigma)] = accuracy
                    valid_iteration = valid_iteration +1
                    
                    '''
                    %matplotlib inline
                    for i in range(3):
                        print clusters_labels[i]
                        gr_dataset()
                        gr_membership_contour(estimated_memberships[i])
                        plt.show()
                    '''
                    
                else:
                    print 'unable to infer an estimated membership function'
                    continue
                            
            else:
                print 'not enough clusters'
                continue
            
        except ValueError as e:
            print str(e)
    
    else:
        print'this permutation has not produced a best couple'
        
        
print 'accuracy for the valid tests', test_accuracy



holdout iteration 0

model selection: trying parameters c=0.1, sigma=0.23
starting clusterization
clusters  [[0, 98, 35, 37, 38, 71, 41, 44, 111, 91, 27, 28, 61, 62], [1], [2], [83, 3, 69, 102, 105, 13, 48, 19, 26, 101, 90, 95], [4], [5], [6], [64, 29, 7, 104, 12, 39, 76, 77, 110, 109, 49, 50, 84, 53, 24, 25, 79, 93, 94], [33, 100, 8, 107, 15, 114, 87], [9], [10], [11], [14], [16], [17], [18], [20], [21], [22], [23], [30], [31], [32], [34], [36], [40], [42], [43], [45], [46], [47], [51], [52], [54], [55], [56], [57], [58], [59], [60], [63], [65], [66], [67], [68], [70], [72], [73], [74], [75], [78], [80], [81], [82], [85], [86], [88], [89], [92], [96], [97], [99], [103], [106], [108], [112], [113], [115], [116], [117], [118], [119]]
not enough clusters

model selection: trying parameters c=1.0, sigma=0.23
starting clusterization
clusters  [[0, 98, 35, 37, 38, 71, 41, 44, 111, 91, 27, 28, 61, 62], [1], [2], [83, 3, 69, 102, 105, 13, 48, 19, 26, 101, 90, 95], [4], [5], [6], [64, 29, 7,

clusters  [[0], [1], [2], [3], [35, 4, 5, 76, 13, 46, 116, 21, 36], [6], [7], [8], [9], [10], [11], [12], [14], [15], [16], [17], [18], [19], [20], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [37], [38], [39], [40], [41], [42], [43], [44], [45], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [117], [118], [119]]
not enough clusters

model selection: trying parameters c=0.1, sigma=0.234
starting clusterization
clusters  [[0, 2, 7, 27, 28, 31, 33, 41, 50, 52, 58, 59, 60, 65, 66, 68, 85, 86, 89, 90, 94, 95, 103, 104, 119], [1], [3], [32, 4, 5, 8, 73, 44, 13, 47, 81, 82, 51, 53, 54, 9, 

clusters  [[0], [1, 3, 5, 7, 11, 16, 20, 31, 41, 50, 52, 55, 64, 65, 74, 87, 94, 95, 97, 98, 107, 112, 113, 114, 118, 119], [2], [66, 36, 54, 71, 40, 9, 12, 77, 110, 15, 80, 104, 47, 22, 88, 4, 58, 59, 28, 30], [6], [8], [10], [13], [14], [17], [18], [19], [21], [23], [24], [25], [26], [27], [29], [32], [33], [34], [35], [37], [38], [39], [42], [43], [44], [45], [46], [48], [49], [51], [53], [56], [57], [60], [61], [62], [63], [67], [68], [69], [70], [72], [73], [75], [76], [78], [79], [81], [82], [83], [84], [85], [86], [89], [90], [91], [92], [93], [96], [99], [100], [101], [102], [103], [105], [106], [108], [109], [111], [115], [116], [117]]
not enough clusters

model selection: trying parameters c=10.0, sigma=0.234
starting clusterization
clusters  [[0], [1, 7, 11, 14, 16, 31, 39, 41, 55, 64, 65, 74, 78, 87, 94, 95, 97, 98, 111, 112, 113, 114, 118], [2], [3], [4, 9, 10, 12, 15, 22, 28, 30, 36, 37, 40, 47, 54, 58, 59, 66, 71, 77, 80, 88, 104, 110], [5], [6], [8], [13], [17], [18], [

clusters  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119]]
not enough clusters

model selection: trying parameters c=1.0, sigma=0.232
starting clusterization
clusters  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26

inferring membership 1
inferring membership 2
unable to infer an estimated membership function

model selection: trying parameters c=1.0, sigma=0.232
starting clusterization
clusters  [[0, 33, 67, 22, 102, 42, 107, 12, 14, 17, 50, 51, 53, 118, 87, 58, 28, 61, 85, 31], [1], [2], [35, 3, 13, 45, 113, 115, 60], [4], [91, 100, 5, 40, 105, 10, 48, 49, 82, 54, 36, 26, 59, 30, 63], [6], [7], [8], [9], [11], [15], [16], [18], [72, 19], [20], [21], [23], [24], [25], [27], [29], [32, 69, 39, 41, 109, 81, 117, 55], [34], [37], [38], [43], [44], [46], [47], [52], [56], [57], [62], [64], [65], [66], [68], [70], [71], [73], [74], [75], [76], [77], [78], [79], [80], [83], [84], [86], [88], [89], [90], [92], [93], [94], [95], [96], [97], [98], [99], [101], [103], [104], [106], [108], [110], [111], [112], [114], [116], [119]]
got enough clusters
labelled clusters  ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
a

...

clusters  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [84], [85], [86], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119]]
not enough clusters

starting training with best couple:  (10.0, 0.234)
starting clusterization
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
va

clusters  [[0], [1], [2], [3, 6, 9, 14, 15, 19, 21, 24, 26, 27, 31, 50, 54, 55, 60, 62, 65, 71, 73, 74, 78, 83, 84, 88, 91, 93, 103, 105, 106, 107, 109, 110, 111, 115, 116], [66, 99, 4, 70, 39, 11, 12, 98, 77, 43, 51, 20, 117, 86, 45, 92, 29], [5], [7], [8], [10], [13], [16], [17], [18], [22], [23], [25], [28], [30], [32], [33], [34], [35], [36], [37], [38], [40], [41], [42], [44], [46], [47], [48], [49], [52], [53], [56], [57], [58], [59], [61], [63], [64], [67], [68], [69], [72], [75], [76], [79], [80], [81], [82], [85], [87], [89], [90], [94], [95], [96], [97], [100], [101], [102], [104], [108], [112], [113], [114], [118], [119]]
not enough clusters

model selection: trying parameters c=1.0, sigma=0.23
starting clusterization
clusters  [[0], [1], [2], [3, 6, 9, 14, 15, 19, 21, 24, 26, 27, 31, 50, 54, 55, 56, 57, 60, 62, 65, 71, 73, 74, 78, 83, 84, 88, 91, 93, 103, 105, 106, 107, 109, 110, 111, 115, 116], [66, 99, 4, 70, 39, 11, 12, 98, 77, 43, 51, 20, 117, 86, 45, 92, 29], [5], [7],

clusters  [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24], [25], [26], [27], [28], [29], [30], [31], [32], [33], [34], [35], [36], [37], [38], [39], [40], [41], [42, 84, 86], [43], [44], [45], [46], [47], [48], [49], [50], [51], [52], [53], [54], [55], [56], [57], [58], [59], [60], [61], [62], [63], [64], [65], [66], [67], [68], [69], [70], [71], [72], [73], [74], [75], [76], [77], [78], [79], [80], [81], [82], [83], [85], [87], [88], [89], [90], [91], [92], [93], [94], [95], [96], [97], [98], [99], [100], [101], [102], [103], [104], [105], [106], [107], [108], [109], [110], [111], [112], [113], [114], [115], [116], [117], [118], [119]]
not enough clusters

model selection: trying parameters c=10.0, sigma=0.23
starting clusterization
clusters  [[0, 35, 100, 49, 114, 54, 24, 30], [1], [2], [3], [97, 34, 67, 4, 101, 38, 71, 42, 110, 80, 81, 102, 84, 86, 89, 56, 25, 90, 29, 16], [5], [6], [7], [8],

inferring membership 1
inferring membership 2
validating model
accuracy 0.933333333333

model selection: trying parameters c=10.0, sigma=0.23
starting clusterization
clusters  [[0], [1], [97, 2, 35, 68, 104, 12, 106, 107, 108, 21, 56, 90, 27, 74, 85], [3, 14, 17, 23, 28, 37, 38, 39, 43, 47, 54, 55, 61, 63, 65, 66, 71, 72, 73, 81, 84, 88, 91, 109, 110, 113], [4], [96, 5], [6], [7], [8], [9], [10], [11], [13], [15], [16], [18], [19], [20], [22], [24], [25], [26], [29], [30], [31], [32], [33], [34], [36], [40], [41], [42], [44], [45], [46], [48], [49], [50], [51], [52], [53], [57], [58], [59], [60], [62], [64], [67], [69], [70], [75], [76], [77], [78], [79], [80], [82], [83], [86], [87], [89], [92], [93], [94], [95], [98], [99], [100], [101], [102], [103], [105], [111], [112], [114], [115], [116], [117], [118], [119]]
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
validating m

inferring membership 1
inferring membership 2
validating model
accuracy 0.933333333333

model selection: trying parameters c=10.0, sigma=0.23
starting clusterization
clusters  [[0], [1], [80, 2, 66, 101, 102, 8, 38, 60, 45, 112, 111, 48, 85, 26, 15, 70, 95], [3], [4], [17, 114, 107, 5, 61], [6], [7], [96, 36, 71, 9, 42, 12, 113, 18, 115, 117, 22, 41, 88, 94, 62], [10], [11], [13], [14], [16], [19], [56, 100, 20, 81], [21], [23], [32, 34, 67, 37, 51, 118, 24, 92], [25], [27], [28], [29], [30], [31], [33], [35], [39], [40], [43], [44], [46], [47], [49], [50], [52], [53], [54], [55], [57], [58], [59], [63], [64], [65], [68], [69], [72], [73], [74], [75], [76], [77], [78], [79], [82], [83], [84], [86], [87], [89], [90], [91], [93], [97], [98], [99], [103], [104], [105], [106], [108], [109], [110], [116], [119]]
got enough clusters
labelled clusters  ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
inferring membership 0
inferring membership 1
inferring membership 2
validating model
acc

In [4]:
print 'accuracy mean for the training set: ', np.array(training_accuracy.values()).mean()
print 'accuracy std deviation for the training set: ', np.array(training_accuracy.values()).std()

print 'accuracy mean for the test set: ', np.array(test_accuracy.values()).mean()
print 'accuracy std deviation for the test set: ', np.array(test_accuracy.values()).std()

accuracy mean for the training set:  0.8481481481481483
accuracy std deviation for the training set:  0.05091750772173156
accuracy mean for the test set:  0.8933333333333333
accuracy std deviation for the test set:  0.08537498983243798
